# Imports

In [1]:
import numpy as np
from glob import glob
import pandas as pd
from astropy.table import Table
from tqdm import tqdm
from dustapprox.models import PolynomialModel,PrecomputedModel,polynomial
from dustapprox.io import svo
from dustapprox.extinction import F99
from pyphot.astropy.sandbox import UnitFilter

# Filter definition (or query from SVO filter service)
Generally just query from SVO (first line).
In my example, I needed to massage the passbands to get the filters I needed.

In [2]:
filt = svo.get_svo_passbands(['HST/ACS_SBC.f115lp','HST/ACS_SBC.f125lp','HST/ACS_SBC.f140lp','HST/ACS_SBC.f150lp','HST/ACS_SBC.f165lp'])
wave = np.concatenate([filt[0].wavelength, filt[1].wavelength, filt[2].wavelength, filt[3].wavelength, filt[4].wavelength])
wave = np.unique(wave)

t115 = np.interp(wave, filt[0].wavelength, filt[0].transmit, left=0, right=0)
t125 = np.interp(wave, filt[1].wavelength, filt[1].transmit, left=0, right=0)
t140 = np.interp(wave, filt[2].wavelength, filt[2].transmit, left=0, right=0)
t150 = np.interp(wave, filt[3].wavelength, filt[3].transmit, left=0, right=0)
t165 = np.interp(wave, filt[4].wavelength, filt[4].transmit, left=0, right=0)

h1 = UnitFilter(wave, t115-t125, name='HST/ACS_SBC.H1')
h2 = UnitFilter(wave, t125-t140, name='HST/ACS_SBC.H2')
h3 = UnitFilter(wave, t140-t150, name='HST/ACS_SBC.H3')
h4 = UnitFilter(wave, t150-t165, name='HST/ACS_SBC.H4')
passbands = [h1, h2, h3, h4]

# Integrate (reddened) model spectra over passbands
This creates the grid for polynomial fitting. Can take some time if using many filters or a large grid
* Errors arise in dustapprox scripts 'extinction.py' and 'svo.py'. Need to fix manually.
* Download the model spectra from SVO, or ask Oren.
* Run this only once- afterwards can read the results from saved file.

In [43]:
models = glob('../SEDer/models/kurucz_spectra/*.fl.dat.txt')
apfields = ['teff', 'logg', 'feh']
extc = F99()
Rv = 3.1
Av = np.concatenate([np.arange(0,0.55,0.1),np.arange(0.55,1.0,0.2),np.arange(1.1,5,0.5)])


logs = []
for fname in tqdm(models):
    data = svo.spectra_file_reader(fname)
    # extract model relevant information
    lamb_unit, flux_unit = svo.get_svo_sprectum_units(data)
    lamb = data['data']['WAVELENGTH'].values * lamb_unit
    flux = data['data']['FLUX'].values * flux_unit
    apvalues = [data[k]['value'] for k in apfields]

    # wavelength definition varies between models
    alambda_per_av = extc(lamb, 1.0, Rv=3.1)

    # Dust magnitudes
    columns = apfields + ['passband', 'mag0', 'mag', 'A0', 'Ax']
    for pk in passbands:
        mag0 = -2.5 * np.log10(pk.get_flux(lamb, flux).value)
        # we redo av = 0, but it's cheap, allows us to use the same code
        for av_val in Av:
            new_flux = flux * np.exp(- alambda_per_av * av_val)
            mag = -2.5 * np.log10(pk.get_flux(lamb, new_flux).value)
            delta = (mag - mag0)
            logs.append(apvalues + [pk.name, mag0, mag, av_val, delta])

logs = pd.DataFrame.from_records(logs, columns=columns)

logs.to_csv('my_grid.csv', index=False)

100%|██████████| 51/51 [00:02<00:00, 20.90it/s]


# Define table
My table is good for 3rd degree polynomial, and for 'passbands' vector that corresponds to a single instruments. For passbands from multiple instruments, or higher degree polynomials, modify this cell.

In [44]:

logs = pd.read_csv('my_grid.csv')
apfields = ['teff','logg','feh']
extc = F99()
Rv = 3.1
Av = np.arange(0,5,0.1)

# Define the table data

data = Table(
    {
        'passband': [pb.name for pb in passbands],
        '1': np.full_like(passbands, np.nan),
        'A0': np.full_like(passbands, np.nan),
        'teffnorm': np.full_like(passbands, np.nan),
        'A0^2': np.full_like(passbands, np.nan),
        'A0 teffnorm': np.full_like(passbands, np.nan),
        'teffnorm^2': np.full_like(passbands, np.nan),  
        'A0^3': np.full_like(passbands, np.nan),
        'A0^2 teffnorm': np.full_like(passbands, np.nan),
        'A0 teffnorm^2': np.full_like(passbands, np.nan),
        'teffnorm^3': np.full_like(passbands, np.nan),
        # 'A0^4': np.full_like(passbands, np.nan),
        # 'A0^3 teffnorm': np.full_like(passbands, np.nan),
        # 'A0^2 teffnorm^2': np.full_like(passbands, np.nan),
        # 'A0 teffnorm^3': np.full_like(passbands, np.nan),
        # 'teffnorm^4': np.full_like(passbands, np.nan),
        'mae': np.full_like(passbands, np.nan),
        'rmse': np.full_like(passbands, np.nan),
        'mean_residuals': np.full_like(passbands, np.nan),
        'std_residuals': np.full_like(passbands, np.nan),
    }
)

# Set column data types explicitly
for col in data.colnames:
    if col != "passband":  # Keep 'passband' as a string
        data[col] = data[col].astype('float64')

# Define metadata (corresponding to your `meta` section)
data.meta = {
    'extinction': {
        'source': 'Fitzpatrick (1999)',
        'R0': [3.1, 3.1],
        'A0': [Av.min(), Av.max()]
    },
    'atmosphere': {
        'source': 'Kurucz (ODFNEW/NOVER 2003)',
        'teff': [int(logs['teff'].min()), int(logs['teff'].max())],
        'logg': [logs['logg'].min(), logs['logg'].max()],
        'feh': [logs['feh'].min(), logs['feh'].max()],
        'alpha': [0.0, 0.0]
    },
    'comment': 'teffnorm = teff / 5040; predicts kx = Ax / A0',
    'model': {
        'kind': 'polynomial',
        'degree': 3,
        'interaction_only': False,
        'include_bias': True,
        'feature_names': ['A0', 'teffnorm']
    }
}

# Fit the grid to polynomials
Error arises in dustapprox polynomial.py, function approx_model. Need to fix manually.

In [5]:
for i,pb in enumerate(passbands):
    fit_res = polynomial.approx_model(r=logs,passband=pb.name,degree=3,input_parameters=['teff','A0'])

    for j in range(1,len(fit_res['coefficients'])):
        data[i][j] = fit_res['coefficients'][j-1]

# Write to an ECSV file
data.write('custom_output.ecsv', format='ascii.ecsv', overwrite=True,delimiter=',')

# Attempt for my own polynomial fit

In [56]:
for i,p in enumerate(passbands):
    passband = p.name
    x = logs[logs['passband']==passband]['A0']
    y = logs[logs['passband']==passband]['teff']/5040
    z = logs[logs['passband']==passband]['Ax']

    # Define the design matrix up to 3th order with cross terms
    xy = x * y
    x2y = (x**2) * y
    xy2 = x * (y**2)

    X = np.vstack([np.ones_like(x),x,y,x**2,xy,y**2,x**3,x2y,xy2,y**3]).T


    # Fit the model using np.linalg.lstsq
    coeffs, _, _, _ = np.linalg.lstsq(X, z, rcond=None)


    # Make predictions
    z_pred = X @ coeffs

    # Calculate residuals and MSE
    residuals = z - z_pred

    # Error metrics
    mae = np.mean(np.abs(residuals))  # Mean Absolute Error
    mse = np.mean(residuals**2)       # Mean Squared Error
    rmse = np.sqrt(mse)               # Root Mean Squared Error
    mean_residuals = np.mean(residuals)
    std_residuals = np.std(residuals)

    # Write coefficients and error metrics to table
    
    for j in range(1,len(coeffs)+1):
        data[i][j] = coeffs[j-1]
    data[i]['mae'] = mae
    data[i]['rmse'] = rmse
    data[i]['mean_residuals'] = mean_residuals
    data[i]['std_residuals'] = std_residuals
    

    # Display results
    # print("Fitted coefficients:", coeffs)
    # print("Mean Absolute Error (MAE):", mae)
    # print("Root Mean Squared Error (RMSE):", rmse)
    # print("Mean of Residuals:", mean_residuals)
    # print("Standard Deviation of Residuals:", std_residuals.round(2))

In [58]:
data.write('custom_output.ecsv', format='ascii.ecsv', overwrite=True,delimiter=',')

# Move output to dustapprox library 

In [59]:
import shutil

old_path = 'custom_output.ecsv'
new_path = '../.venv/Lib/site-packages/dustapprox/data/precomputed/polynomial/f99/kurucz/hst_kurucz_f99_a0_teff.ecsv'

shutil.move(old_path, new_path)

'../.venv/Lib/site-packages/dustapprox/data/precomputed/polynomial/f99/kurucz/hst_kurucz_f99_a0_teff.ecsv'

# Verify readable

In [60]:
lib = PrecomputedModel()
lib.find('hst')

[{'extinction': {'A0': [0.0, 4.9],
   'R0': [3.1, 3.1],
   'source': 'Fitzpatrick (1999)'},
  'atmosphere': {'alpha': [0.0, 0.0],
   'feh': [0.0, 0.0],
   'logg': [4.0, 4.0],
   'source': 'Kurucz (ODFNEW/NOVER 2003)',
   'teff': [3500, 25000]},
  'comment': 'teffnorm = teff / 5040; predicts kx = Ax / A0',
  'model': {'degree': 3,
   'feature_names': ['A0', 'teffnorm'],
   'include_bias': True,
   'interaction_only': False,
   'kind': 'polynomial'},
  'passbands': ['HST/ACS_SBC.H1',
   'HST/ACS_SBC.H2',
   'HST/ACS_SBC.H3',
   'HST/ACS_SBC.H4'],
  'filename': 'c:\\Users\\oreni\\Documents\\code\\SED\\.venv\\Lib\\site-packages\\dustapprox\\data\\precomputed\\polynomial\\f99\\kurucz\\hst_kurucz_f99_a0_teff.ecsv'}]